# Algorithmes génétiques

Vous souhaitez partir dans l'espace et trouvez un manuel un peu étrange. Ce manuel contient un message en 32 caractères qui n'est plus lisible. Le manuel vous fournit en revanche une fonction Python (!) qui, à partir d'un message passé en paramètre (chaîne de caractère) renvoit le nombre de caractères correctement placés par rapport au message d'origine.

In [1]:
from stochastic.data import score

In [2]:
score("Hi guys!")

1

Bon, c'est déjà ça...

La fonction fournie permet de faire des tests avec une autre solution, ce qui va nous permettre de mettre au point un algorithme de résolution.

In [3]:
score("plop", solution="ploc")

3

Nous allons mettre au point un algorithme de résolution de type « algorithmes génétiques » pour résoudre le décodage du message "Hello world!".

In [4]:
score_hello = lambda x: score(x, solution="Hello world!")
score_hello("Hello world!")

12

Tout d'abord, considérons l'ensemble des caractères qui forment notre mot. On a le droit:
 - aux vingt-six lettres de l'alphabet minuscules; (message en anglais, sans accent!)
 - aux mêmes lettres en majuscules;
 - à la ponctuation.

In [5]:
import string

letters = string.ascii_uppercase + string.ascii_lowercase + string.punctuation + ' '
letters

'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

Puisqu'on travaille avec un problème plus petit, on stocke cette taille:

In [6]:
length = score_hello("Hello world!")
length

12

La bibliothèque `random` nous sera utile pour cette séance. Elle propose notamment la fonction `choice`:

In [7]:
import random

random.choice(letters)

'&'

On peut également tirer plusieurs lettres (différentes) avec la fonction `sample`:

In [8]:
random.sample(letters, 3)

['(', 'O', ')']

Notons également les deux fonctions suivantes pour transformer une chaîne de caractère en liste, et inversement:

In [9]:
list("toto")

['t', 'o', 't', 'o']

In [10]:
"".join([random.choice(letters) for _ in range(12)])

'-%M]%/`MeDC*'

Estimons le temps d'évaluation de la fonction `score_hello`:

In [11]:
%%timeit
score_hello("Hello,World~")

1.96 µs ± 51.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


<div class="alert alert-warning">
<b>Question :</b> Estimer le temps d'évaluation au pire des cas (bruteforce) de tous les messages à 12 (puis à 32) caractères possibles.
</div>

In [ ]:
1.7e-6 * len(letters)**12

In [ ]:
# en nombre d'années (âge de l'univers: 1.38e10)
"12 letters: {:.3g} yr; 32 letters: {:.3g} yr".format(
    1.7e-6 * len(letters)**12 / 365 / 24 / 60 / 60,
    1.7e-6 * len(letters)**32 / 365 / 24 / 60 / 60)

## Algorithmes

<div class="alert alert-warning">
<b>Théorie !</b>
</div>

Voir les slides...

<div class="alert alert-warning">
<b>En pratique...</b>
</div>

Toute la difficulté dans l'utilisation des algorithmes génétiques revient à correctement choisir un relativement grand nombre de paramètres:

 - comment choisir une taille de la population de départ ;
 - comment initialiser la population de départ ;
 - comment procéder aux croisements :
     - comment choisir deux éléments à croiser (*la sélection*);
     - comment croiser les éléments ;
 - comment procéder aux mutations :
     - quel taux de mutation choisir ;
     - comment muter un élément ;
 - comment arrêter la recherche :
     - on peut fixer un nombre d'itérations maximal ;
     - comment s'assurer qu'on conserve toujours la meilleure instance (*l'élitisme*) ;
 - comment optimiser la convergence :
     - la distribution (détails en annexe pour les personnes motivées/intéressées/en avance).
     
<div class="alert alert-success">
<b>Objectifs de la séance :</b> La suite de l'exercice consiste à coder des algorithmes génétiques en utilisant votre inspiration pour essayer différents opérateurs de sélection, de croisement et de mutation.
</div>

Quelques remarques :
 1. **Nous sommes là pour vous guider**, pour vous suggérer des pistes d'amélioration, mais aussi pour vous laisser faire vos erreurs/comprendre par vous-même pourquoi une méthode n'est pas forcément pertinente;
 1. En paramétrant des méthodes stochastiques, on traverse en général une longue phase de « ça ne fonctionne pas » avant d'arriver aux bons paramètres qui permettent de résoudre le problème de manière efficace à tous les coups;
 1. Essayez de **garder une interface générique** pour vos fonctions afin de pouvoir facilement remplacer les opérateurs que vous testerez.
 
<div class="alert alert-warning">
<b>C'est à vous !</b>
</div>

<div class="alert alert-success">
<b>Résolution :</b> Essayons maintenant avec la fonction reçue par notre ami !
</div>

Il faudra sans doute rejouer avec différents paramètres de l'algorithme pour trouver une solution...

Dans le pire des cas, il faut garder à l'esprit la philosophie des méthodes stochastiques, à savoir « Mieux vaut une solution pas trop mauvaise que pas de solution ».

## Annexe : calcul distribué, fonctionnement par îlots.

Une manière de distribuer les calculs quand on est :
 - un peu limite en ressource ;
 - coincé dans des minima locaux ;
 
consiste à lancer plusieurs exécutions du même algorithme en parallèle. Cette méthode permet également d'**avoir un comportement plus stable d'une exécution à l'autre**.

Chaque algorithme va alors converger vers différents minima locaux. Le principe des îlots consiste alors à faire voyager les meilleurs éléments de chaque îlot vers les îlots voisins afin qu'ils se croisent avec d'autres populations. Il faut alors trouver un rythme de *voyage* qui permette à chaque îlot de développer des spécificités tout en brassant suffisamment souvent pour aider à la convergence.

Nous vous proposons alors le code suivant à base de threads (module `concurrent.futures`) et avec des queues (thread-safe!) pour communiquer. Les particularités du langage Python (rechercher "Global Interpreter Lock" (GIL) pour plus de détails...) ne permettent pas de procéder à un vrai multithreading donc la méthode serait à vrai dire plus efficace dans un autre langage de programmation.